In [0]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler as ROS
from imblearn.over_sampling import SMOTE as SMO

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train =pd.read_csv('/content/gdrive/My Drive/ML/Training_x.csv',sep='\t', encoding='utf-8')
test = pd.read_csv('/content/gdrive/My Drive/ML/Testing_x.csv',sep='\t', encoding='utf-8')


In [0]:
data = pd.concat([train,test],axis = 0)
data = data.reset_index()
data = data.drop(columns = ['index'])

In [0]:

data['ArrDelay'] = 0

def func(row):
    if row['ArrDel15']>0:# consider 15
        return 1
    else:
        return 0
data['ArrDelay'] = data.apply(func, axis=1)

In [0]:
x_tr = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek','CRSArrTime',
       'FlightNum', 'Origin', 'DestAirportID', 'Dest','year','DepDelay',
       'DewPointC', 'DewPointF', 'FeelsLikeC', 'FeelsLikeF', 'HeatIndexC',
       'HeatIndexF', 'WindChillC', 'WindChillF', 'WindGustKmph',
       'WindGustMiles', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'tempF', 'time', 'visibility', 'weatherCode', 'winddir16Point',
       'winddirDegree', 'windspeedKmph', 'windspeedMiles']
X= data[x_tr]
x_val = data[x_tr].iloc[:,:].values

In [0]:
train['Origin'].unique()

array(['PHX', 'ATL', 'DEN', 'LAX', 'CLT', 'ORD', 'JFK', 'LAS', 'MCO',
       'DFW', 'MIA', 'EWR', 'IAH', 'SEA', 'SFO'], dtype=object)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc  = LabelEncoder()
enc.fit(x_val[:,8].reshape(-1, 1) )
c1=['d1']
temp = pd.DataFrame(enc.transform(x_val[:,6]),columns = c1)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Origin'])
c2=['a1']
temp = pd.DataFrame(enc.transform(x_val[:,8]),columns = c2)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Dest'])
X = X.drop(columns = ['winddir16Point'])
X = X.drop(columns = ['DewPointF','FeelsLikeC','FeelsLikeF','HeatIndexF','WindChillF','WindGustMiles','tempF','windspeedMiles','Quarter'])
x_val = X.iloc[:,:].values
y = data['ArrDelay']
y_val  = y.iloc[:].values

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
sampler = SMO()
X,y =  sampler.fit_resample(x_val,y_val)

In [0]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
from sklearn.model_selection import train_test_split 
X_train1, X_test1, y_train1, y_test1 = train_test_split(x_val, y_val, test_size=0.33, random_state=42)
sampler = SMO()
X_train1,y_train1 =  sampler.fit_resample(X_train1,y_train1)

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators=10)
classifier2.fit(X_train1,y_train1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
print(classifier2.score(X_test1, y_test1, sample_weight=None))
from sklearn.metrics import classification_report
y_pred = classifier2.predict(X_test1)
print(classification_report(y_true=y_test1,y_pred = y_pred))

0.912112169668851
              precision    recall  f1-score   support

           0       0.93      0.97      0.95    478057
           1       0.85      0.71      0.77    127169

    accuracy                           0.91    605226
   macro avg       0.89      0.84      0.86    605226
weighted avg       0.91      0.91      0.91    605226



In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators=10)
classifier2.fit(X_train,y_train)
print(classifier2.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred = classifier2.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred))

0.9386535964928785
              precision    recall  f1-score   support

           0       0.92      0.96      0.94    478354
           1       0.96      0.91      0.94    478328

    accuracy                           0.94    956682
   macro avg       0.94      0.94      0.94    956682
weighted avg       0.94      0.94      0.94    956682



In [0]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
classifier3 = GBC(n_estimators= 30)
classifier3.fit(X_train1,y_train1)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=30,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
print(classifier3.score(X_test1, y_test1, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier3.predict(X_test1)
print(classification_report(y_true=y_test1,y_pred = y_pred1))

0.8952110451302489
              precision    recall  f1-score   support

           0       0.94      0.93      0.93    478057
           1       0.74      0.78      0.76    127169

    accuracy                           0.90    605226
   macro avg       0.84      0.85      0.84    605226
weighted avg       0.90      0.90      0.90    605226



In [0]:
classifier3 = GBC(n_estimators= 30)
classifier3.fit(X_train,y_train)
print(classifier3.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier3.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred))

0.8955075981360577
              precision    recall  f1-score   support

           0       0.92      0.96      0.94    478354
           1       0.96      0.91      0.94    478328

    accuracy                           0.94    956682
   macro avg       0.94      0.94      0.94    956682
weighted avg       0.94      0.94      0.94    956682



In [0]:
from sklearn.ensemble import ExtraTreesClassifier as ETC
classifier4 = ETC(n_estimators = 30)
classifier4.fit(X_train1,y_train1)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [0]:
print(classifier4.score(X_test1, y_test1, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier4.predict(X_test1)
print(classification_report(y_true=y_test1,y_pred = y_pred1))

0.9020663355506868
              precision    recall  f1-score   support

           0       0.91      0.97      0.94    478057
           1       0.84      0.66      0.74    127169

    accuracy                           0.90    605226
   macro avg       0.88      0.81      0.84    605226
weighted avg       0.90      0.90      0.90    605226



In [0]:
from sklearn.ensemble import ExtraTreesClassifier as ETC
classifier4 = ETC(n_estimators = 30)
classifier4.fit(X_train,y_train)
print(classifier4.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier4.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred1))

0.9428096274415114
              precision    recall  f1-score   support

           0       0.92      0.97      0.94    478354
           1       0.96      0.92      0.94    478328

    accuracy                           0.94    956682
   macro avg       0.94      0.94      0.94    956682
weighted avg       0.94      0.94      0.94    956682



In [0]:
from sklearn import svm
classifier5 = svm.SVC()
classifier5.fit(X_train1,y_train1)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
print(classifier5.score(X_test1, y_test1, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier5.predict(X_test1)
print(classification_report(y_true=y_test1,y_pred = y_pred1))

In [0]:
from sklearn import svm
classifier5 = svm.SVC()
classifier5.fit(X_train,y_train)
print(classifier5.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier5.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred1))

In [0]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth  = 8, base_score = 0.5,learning_rate = 0.2)
model.fit(X_train1, y_train1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test1)
accuracy = accuracy_score(y_test1, y_pred)
from sklearn.metrics import classification_report
print("Accuracy: %.2f%%" % (accuracy * 100.0))
y_pred1 = model.predict(X_test1)
print(classification_report(y_true=y_test1,y_pred = y_pred1))

Accuracy: 90.43%
              precision    recall  f1-score   support

           0       0.94      0.94      0.94    478057
           1       0.78      0.75      0.77    127169

    accuracy                           0.90    605226
   macro avg       0.86      0.85      0.85    605226
weighted avg       0.90      0.90      0.90    605226



In [0]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth  = 8, base_score = 0.5,learning_rate = 0.2)
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
y_pred1 = model.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred))

Accuracy: 94.13%
              precision    recall  f1-score   support

           0       0.92      0.96      0.94    478354
           1       0.96      0.92      0.94    478328

    accuracy                           0.94    956682
   macro avg       0.94      0.94      0.94    956682
weighted avg       0.94      0.94      0.94    956682

